In [ ]:
!nvidia-smi

In [ ]:
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision 
from torchvision import datasets, transforms 
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [ ]:
# device agnostic code

device = "cuda" if torch.cuda.is_available() else "cpu"

device

In [ ]:
# getting dataset

# !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

# wget did not work for this kaggle notebook, hence i manually downloaded the dataset and uploaded it to kaggle

In [ ]:
# loading train data

TRAINING_PATH = "tiny-imagenet-200/train"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

training_data = datasets.ImageFolder(root = TRAINING_PATH, transform = transform, target_transform = None)

In [ ]:
# loading validation data

VAL_PATH = "tiny-imagenet-200/val"


with open("tiny-imagenet-200/val/val_annotations.txt") as f:
    lines = f.readlines()
    
val_dict = {}

for line in lines:
    parts = line.strip().split('\t')
    val_dict[parts[0]] = parts[1]
    
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
    

val_data = datasets.ImageFolder(root = VAL_PATH, transform = transform, target_transform = None)

for i in range(len(val_data)):
    img_path, _ = val_data.imgs[i]
    img_name = os.path.basename(img_path)
    val_data.imgs[i] = (img_path, training_data.classes.index(val_dict[img_name]))

In [ ]:
# data shapes

print(f"Length of training data = {len(training_data)}, Shape of Image = {training_data[0][0].shape}")
print(f"Length of validation data = {len(val_data)}, Shape of Image = {val_data[0][0].shape}")

In [ ]:
# label mapping file
with open("tiny-imagenet-200/words.txt", 'r') as f:
    class_names = f.readlines()
    
# mapping between WordNet IDs to class names
class_dict = {}
for line in class_names:
    line = line.split('\t')
    class_dict[line[0]] = line[1].strip()
    
    
# visualise
torch.manual_seed(1234)
fig = plt.figure(figsize=(12, 12))
rows, cols = 3, 3
for i in range(rows * cols):
    rand_idx = torch.randint(0, len(training_data), size = [1]).item()
    image, target = training_data[rand_idx]
    fig.add_subplot(rows, cols, i + 1)
    plt.imshow(image.permute(1, 2, 0))
    plt.title(class_dict[training_data.classes[target]])
    plt.axis(False)

In [ ]:
# dataloader

BATCH_SIZE = 32

training_dataloader = DataLoader(dataset = training_data, batch_size = BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(dataset = val_data, batch_size = BATCH_SIZE, shuffle = True)

training_images, training_targets = next(iter(training_dataloader))

print(f"Training images batch shape = {training_images.shape}")
print(f"Training targets batch shape = {training_targets.shape}")

In [ ]:
# DenseNet


class DenseNet(nn.Module):
    def __init__(self, config, num_classes):
        super().__init__()

        self.block_config = {
            121 : [6, 12, 24, 16],
            169 : [6, 12, 32, 32],
            201 : [6, 12, 48, 32],
            264 : [6, 12, 64, 48]
        }

        self.k = 32

        self.bn1 = nn.BatchNorm2d(3)
        self.relu = nn.ReLU()
        self.conv_1 = nn.Conv2d(in_channels = 3, out_channels = 2 * self.k, kernel_size = 7, stride = 2, padding = 3)
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)

        self.dense_blocks = nn.ModuleList([])
        self.k_0 = 2 * self.k

        for i, l in enumerate(self.block_config[config]):
            self.dense_blocks.append(dense_block(num_layers = l, in_channels = self.k_0, growth_rate = self.k))
            self.k_0 += self.k * l

            if i != len(self.block_config[config]) - 1:
                self.dense_blocks.append(transition_layer(in_channels = self.k_0, theta = 0.5))
                self.k_0 //= 2

        self.flatten = nn.Flatten()
        self.bn2 = nn.BatchNorm2d(self.k_0)
        self.global_avgpool = nn.AvgPool2d(kernel_size = 7)
        self.fc = nn.Linear(in_features = self.k_0, out_features = num_classes)


        self.init_weights()


    def init_weights(self):
        for layer in self.modules():
            if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
                nn.init.kaiming_uniform_(layer.weight)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias)
                

    def forward(self, x):
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv_1(x)
        x = self.maxpool(x)

        for layer in self.dense_blocks:
            x = layer(x)

        x = self.bn2(x)
        x = self.relu(x)
        x = self.global_avgpool(x)
        x = self.flatten(x)
        return self.fc(x)



class dense_block(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super().__init__()

        self.block = nn.ModuleList([])

        for i in range(num_layers):
            self.block.append(dense_layer(in_channels = in_channels + (i * growth_rate), growth_rate = growth_rate))


    def forward(self, x):
        for layer in self.block:
            x = layer(x)
        return x



class transition_layer(nn.Module):
    def __init__(self, in_channels, theta):
        super().__init__()

        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU()
        self.conv_1x1 = nn.Conv2d(in_channels = in_channels, out_channels = int(theta * in_channels), kernel_size = 1, stride = 1, padding = 0)
        self.avgpool = nn.AvgPool2d(kernel_size = 2, stride = 2)

    def forward(self, x):
        x = self.bn(x)
        x = self.relu(x)
        x = self.conv_1x1(x)
        return self.avgpool(x)



class dense_layer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super().__init__()

        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU()
        self.conv_1x1 = nn.Conv2d(in_channels = in_channels, out_channels = 4 * growth_rate, kernel_size = 1, stride = 1, padding = 0)
        self.bn2 = nn.BatchNorm2d(4 * growth_rate)
        self.conv_3x3 = nn.Conv2d(in_channels = 4 * growth_rate, out_channels = growth_rate, kernel_size = 3, stride = 1, padding = 1)

    def forward(self, x):
        feature_maps = x

        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv_1x1(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv_3x3(x)

        return torch.cat([feature_maps, x], 1)

In [ ]:
# train and val metrics 

train_loss_values = []
val_loss_values = []
val_acc_values = []
epoch_count = []

In [ ]:
# training loop

def model_train(epochs, model, train_dataloader, val_dataloader, loss_func, optimizer, scheduler):

    # turn on training mode
    model.train()

    #check training device
    print(f"Training on {device}.")

    # loop through each epoch
    for epoch in range(epochs):
        print(f"Epoch: {epoch + 1}/{epochs}\n-------------")

        # loop through each batch
        train_loss, train_acc = 0, 0
        total_steps = 1
        for images, classes in train_dataloader:

            #send data to device
            images, classes = images.to(device), classes.to(device)

            # computer forward pass
            y_pred = model(images)

            # compute loss
            loss = loss_func(y_pred, classes)
            train_loss += loss
            train_acc += accuracy_fn(y_true = classes, y_pred = y_pred.argmax(dim=1))

            # update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_loss = train_loss / total_steps
            batch_acc = train_acc / total_steps

            if total_steps % 10 == 0:
                print(f"Training Loss: {batch_loss:.5f} - Training Accuracy: {batch_acc:.5f}%")

            total_steps += 1

        # learning rate decay
        scheduler.step()


        # performance on test set
        # turn on inference mode
        with torch.inference_mode():
            # loop through each batch
            total_val_loss, val_acc = 0, 0
            for val_images, val_classes in val_dataloader:
                # send data to device
                val_images, val_classes = val_images.to(device), val_classes.to(device)

                # forward pass
                y_val_pred = model(val_images)

                # compute loss
                val_loss = loss_func(y_val_pred, val_classes)
                total_val_loss += val_loss
                val_acc += accuracy_fn(y_true = val_classes, y_pred = y_val_pred.argmax(dim=1)
                )
            
            total_val_loss /= len(val_dataloader)
            val_acc /= len(val_dataloader)


        train_loss /= len(train_dataloader)
        train_acc /= len(train_dataloader)

        print(f"[After {epoch + 1} epochs: Train Loss: {train_loss:.5f} - Train Accuracy: {train_acc:.5f}% - Validation Loss: {total_val_loss:.5f} - Validation Accuracy: {val_acc:.5f}%]")

        
        train_loss_values.append(train_loss.item())
        val_loss_values.append(total_val_loss.item())
        val_acc_values.append(val_acc)
        epoch_count.append(epoch + 1)

In [ ]:
# test loop

def model_test(model, dataloader, loss_func):
    # turn on test mode
    model.eval()
    
    # turn on inference mode
    with torch.inference_mode():
        # loop through each batch
        test_loss, test_acc = 0, 0
        for images, classes in dataloader:
            # send data to device
            images, classes = images.to(device), classes.to(device)

            # forward pass
            y_pred = model(images)

            # compute loss
            loss = loss_func(y_pred, classes)
            test_loss += loss
            test_acc += accuracy_fn(y_true = classes, y_pred = y_pred.argmax(dim=1)
            )
        
        test_loss /= len(dataloader)
        test_acc /= len(dataloader)
        print(f"Loss: {test_loss:.5f} - Accuracy: {test_acc:.5f}%")
        return test_acc

In [ ]:
# metric functions

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
# instantiating model

torch.manual_seed(1234)
densenet = DenseNet(num_classes = 200, config = 121).to(device)

In [ ]:
# loss function and optimizer

LEARNING_RATE = 0.1

loss_func = nn.CrossEntropyLoss()

sgd = torch.optim.SGD(params = densenet.parameters(), lr = LEARNING_RATE, momentum = 0.9, weight_decay = 0.0001)

learning_decay = torch.optim.lr_scheduler.StepLR(optimizer = sgd, step_size = 30, gamma = 0.1)

In [ ]:
# training the model

EPOCHS = 90

torch.manual_seed(1234)
model_train(epochs = EPOCHS, model = densenet, train_dataloader = training_dataloader, val_dataloader = val_dataloader, loss_func = loss_func, optimizer = sgd, scheduler = learning_decay)

In [ ]:
# evaluating the model

accuracy = model_test(model = resnet, dataloader = val_dataloader, loss_func = loss_func)

In [ ]:
# loss curve

plt.figure(figsize=(13, 7))
plt.plot(epoch_count, train_loss_values, label = "Train loss")
plt.plot(epoch_count, val_loss_values, label = "Validation loss")
plt.title("Loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
# accuracy curve

plt.figure(figsize=(13, 7))
plt.plot(epoch_count, val_acc_values, label = "Accuracy")
plt.title("Accuracy curves")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
# saving the model

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents = True, exist_ok = True)

MODEL_NAME = "DenseNet_" + str(accuracy).replace(".", "_") + ".pth"

MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving DenseNet to {MODEL_SAVE_PATH}")
torch.save(obj = dense.state_dict(), f = MODEL_SAVE_PATH)